In [12]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    [[ -a myfile ]] && rm myfile
    [[ -a errors ]] && rm errors 
    [[ -a mydate ]] && rm mydate
    [[ -a mynewdir ]] && rm -rf mynewdir
    [[ -a anotherfile ]] && rm anotherfile
    [[ -a mybin ]] && rm -rf  mybin
    [[ -a myinfo ]] && rm myinfo''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()
bash = BashSession()

generated="~/myfile ~/errors ~/mydate ~/mydir ~/mynewdir ~/out"


Exception ignored in: <function BashSession.__del__ at 0x7fc13fcaa200>
Traceback (most recent call last):
  File "/opt/app-root/src/openos/content/python/common.py", line 868, in __del__
    bashSessionClose(self.session)
  File "/opt/app-root/src/openos/content/python/common.py", line 847, in bashSessionClose
    closeTtySession(session)
  File "/opt/app-root/src/openos/content/python/common.py", line 618, in closeTtySession
    TtyOpenSessions.remove(session)
ValueError: list.remove(x): x not in list


```{warning}
This portion of the book is under construction, not ready to be read
```

(cont:gs:abstractions)=
# Operating System Abstractions

UNIX has instilled in generations of computer scientists a basic aesthetic for how to design and structure complicated collections of software.  In particular one learns that the designers of UNIX tried to structure the system around a small core set of ideas, "abstractions", that once understood allows a programmer to understand the rest of the system and how to get things done.   In this chapter we introduce some of those key abstractions. 



## Files

Probably the most novel idea of Unix is that everything is a file, where a file is a stream of bytes.  A process can use files to access durable state on a disk, communicate with other processes, find out information about the system, and communicate over the network.   This central idea enables one to write programs that do the same task irrespective of where the data is coming from or going to.  As we will see, it also allows programmers to tie together multiple simple programs to do a more complex task.  


```{figure} ../images/intro/file-desc.png
---
width: 50%
name: file-desc
---
The kernel array of file descriptors.
```

### File Descriptors
As shown {numref}`file-desc` the kernel maintains a table of open files, and a file descriptor is an index into that table that can be used to read or write to a particular file.  Entries are added to the table by operations that open or create a file or create a special file like a network connection.  There are three special file descriptors shown in {numref}`fd-table`, that programs (and libraries) should use for input, output and errors. 

```{list-table} Standard well known file descriptors. 
:header-rows: 1
:name: fd-table
:width: 50%

* - Value
  - Name
  - Purpose
* - 0
  - stdin
  - standard input; process should read data from here
* - 1
  - stdout
  - standard output; process will write its output here
* - 2
  - stderr
  - standard error; process should write out errors here
```

The two most important operations on files are:

- **ssize_t read(int fd, void *buf, size_t count)**: read from file referenced by descriptor *fd* into memory pointed to by *buf* as up to *count* bytes.
- **ssize_t write(int fd, void *buf, size_t count)**: write from file referenced by descriptor *fd* from memory pointed to by *buf*, up to *count* bytes.

To understand how to use these operations, you really need to read the manual.  In Linux you can find out about everything using the **man** program.  For example:

In [13]:
bash.run("man 2 write", height='1in')

Output(layout=Layout(border='1px solid black', height='1in', overflow_y='scroll'))

```{note}
:class: margin
In this case, the *2* refers to the section of the manual for system calls.  To find out about the different sections, you, of course, read the manual about the man command.
```



### Composing programs together

So what's the big deal, well it turns out that if we can read and write streams, we can now write programs that combine together to do something very powerful.  For example, the **|** symbol tells the shell to take the output of one program and send it to the input of the next program.  So, lets say we are trying to find all the programs on our computer that have anything to do with perl, the following command will list the contents of the /usr/bin/ directory send the output of that listing to a grep program that searches for the word perl, and send the output of that to a program that counts the number of lines of input it had


In [18]:
bash.run("ls /usr/bin/ | grep perl | wc -l")

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

Linux now exposes all kinds of information through synthetic file systems, giving users and administrators massive ability to automate.  For example, in ```bash``` the shell we are using ```$$``` lets us know the ``id`` of the process we are in.  So, stealing a nice example from [jonathan](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#standard-output-and-redirection), the following command shows the file descriptors for stdin, stdout, and stderr. 

In [15]:
bash.run('''ls -l /proc/$$/fd/{0..2}''')

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

In [16]:
bash_pid=bash.getPid()
bash_stdout=os.path.realpath("/proc/" + bash_pid.__str__() + "/fd/1")
bash.run("file " + bash_stdout)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

And we can see that our stdin, stdout, and stderr all point to a character special file, which in Unix is used to represent a terminal.  If we echo hello to that same special device, it will appear in our terminal. 

In [17]:
bash.run("echo \"hello class\" > " + bash_stdout)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

The goal of this section was to discuss some of the power of the shell and the abstraction Unix has of everything being a file.  I would strongly encourage reading the shell and unix sections of [Under the Covers: The Secret Life of Software](https://jappavoo.github.io/UndertheCovers/textbook/intro_tb.html#under-the-covers-the-secret-life-of-software)  for a much more detailed coverage of this material.